#depth calo
X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

depth=[3*X0_ECAL, 16*X0_ECAL, 6*X0_ECAL, 1.5*Lambda_int_HCAL, 4.1*Lambda_int_HCAL, 1.8*Lambda_int_HCAL ]
#depth=[ 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]
print(depth)

# Predictions

In [1]:
import h5py
import numpy as np
import ROOT

Welcome to JupyROOT 6.16/00


## Importing Files
Importing Both
 - file containing the energy [MeV] in each cell:
 
       Calo with 6 layers (3 ECAL: [3, 16, 6]X0_E 3 HCAL: [1.5, 4.1, 2.8]lI_H ) with [64,32,32,16,16,8]^2 pixel 
       
       Dimensions: Beam distance 150 cm, Transversal length 125x125 cm^2, 
           [11.691, 62.352, 23.382, 26.157, 71.4958, 31.3884] cm  
       
          Rad. Len.: X0_E = 3.897  X0_H = 2.357
          
          Int. Len.: lI_E = 37.578 lI_H = 17.438
 
 - file containing the topocluster information. Int number for each cell associated to the topocluster


In [2]:
#Energy Calo
f = h5py.File('../File_En/Outfile_CellInformation_HomDet_15to20GeV_FineGran128_V1.h5','r')
fN= h5py.File('../File_En/Outfile_MLPred_HomDet_15to20GeV_FineGran128_V1.h5','r')

f = h5py.File('../File_En/Outfile_CellInformation_HomDet_2to5GeV_FineGran128.h5','r')
EV =10
layers=[64,32,32,16,16,8]
no=[13,34,17,54,33,54]
for lay in range(6):
    T=f['RealRes_ChargedEnergy_Layer'+str(lay+1)][EV][0]+f['RealRes_Noise_Layer'+str(lay+1)][EV][0]
    for X in range(layers[lay]):
        for Y in range(layers[lay]):
            if T[X][Y]>5*no[lay]: print(X,Y, T[X][Y], lay)

In [3]:
print(f.keys())

<KeysViewHDF5 ['MidPoint_X_Layer1', 'MidPoint_X_Layer2', 'MidPoint_X_Layer3', 'MidPoint_X_Layer4', 'MidPoint_X_Layer5', 'MidPoint_X_Layer6', 'MidPoint_Y_Layer1', 'MidPoint_Y_Layer2', 'MidPoint_Y_Layer3', 'MidPoint_Y_Layer4', 'MidPoint_Y_Layer5', 'MidPoint_Y_Layer6', 'MidPoint_Z_Layer', 'Photon1_E', 'Photon1_Phi', 'Photon1_Theta', 'Photon2_E', 'Photon2_Phi', 'Photon2_Theta', 'Pi0_Phi', 'Pi0_Theta', 'RealRes_ChargedEnergy_Layer1', 'RealRes_ChargedEnergy_Layer2', 'RealRes_ChargedEnergy_Layer3', 'RealRes_ChargedEnergy_Layer4', 'RealRes_ChargedEnergy_Layer5', 'RealRes_ChargedEnergy_Layer6', 'RealRes_NeutralEnergy_Layer1', 'RealRes_NeutralEnergy_Layer2', 'RealRes_NeutralEnergy_Layer3', 'RealRes_NeutralEnergy_Layer4', 'RealRes_NeutralEnergy_Layer5', 'RealRes_NeutralEnergy_Layer6', 'RealRes_Noise_Layer1', 'RealRes_Noise_Layer2', 'RealRes_Noise_Layer3', 'RealRes_Noise_Layer4', 'RealRes_Noise_Layer5', 'RealRes_Noise_Layer6', 'RealRes_TotalEnergy_Layer1', 'RealRes_TotalEnergy_Layer2', 'RealRes_To

In [4]:
#cluster on total
gT = h5py.File('../File_Clust/Outfile_15to20GeV_TotalTopo.h5','r')

#cluster on charge wo noise
gCh= h5py.File('../File_Clust/Outfile_15to20GeV_ChargedTopo.h5','r')

## Calorimeter parameters (X0, lambda_I, ...)

In [5]:
import math

TrDi=125.

X0_ECAL = 3.897
Lambda_int_HCAL = 17.438

Total_ECAL_Length = 3 * X0_ECAL + 16 * X0_ECAL + 6 * X0_ECAL
Total_HCAL_Length = 1.5 * Lambda_int_HCAL + 4.1 * Lambda_int_HCAL + 1.8 * Lambda_int_HCAL
Total_Calo_Length = Total_ECAL_Length + Total_HCAL_Length + 1.0 # -- there is a 1 cm gap between ECAL & HCAL


zpos_ECAL1 = -1 * Total_Calo_Length/2 + 3 * X0_ECAL/2
zpos_ECAL2 = zpos_ECAL1 + 3 * X0_ECAL/2 + 16  * X0_ECAL/2
zpos_ECAL3 = zpos_ECAL2 + 16 * X0_ECAL/2 + 6 * X0_ECAL/2

zpos_GAP = zpos_ECAL3 + 6 * X0_ECAL/2 + 1/2

zpos_HCAL1 = zpos_GAP + 1/2 + 1.5 * Lambda_int_HCAL/2
zpos_HCAL2 = zpos_HCAL1 + 1.5 * Lambda_int_HCAL/2 + 4.1 * Lambda_int_HCAL/2
zpos_HCAL3 = zpos_HCAL2 + 4.1 * Lambda_int_HCAL/2 + 1.8 * Lambda_int_HCAL/2

layer_struct = {'layer1' : 64, 'layer2' : 32, 'layer3' : 32, 'layer4' : 16, 'layer5' : 16, 'layer6' : 8}

def MakeTruthTrajectory(Theta, Phi, x_indx, y_indx) : 

    x_orig, y_orig = 0., 0.
    z_orig = -1 * (Total_Calo_Length/2 + 150.)

    if(x_indx == 5 and y_indx == 5) : x_orig, y_orig   =  10.,  10.
    if(x_indx == 5 and y_indx == -5) : x_orig, y_orig  =  10., -10.
    if(x_indx == -5 and y_indx == 5) : x_orig, y_orig  = -10.,  10.
    if(x_indx == -5 and y_indx == -5) : x_orig, y_orig = -10., -10.

    layer1_x = x_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer1_y = y_orig + (zpos_ECAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer2_x = x_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer2_y = y_orig + (zpos_ECAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer3_x = x_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer3_y = y_orig + (zpos_ECAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer4_x = x_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer4_y = y_orig + (zpos_HCAL1 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer5_x = x_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer5_y = y_orig + (zpos_HCAL2 - z_orig) * math.tan(Theta) * math.sin(Phi)

    layer6_x = x_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.cos(Phi)
    layer6_y = y_orig + (zpos_HCAL3 - z_orig) * math.tan(Theta) * math.sin(Phi)

    CellIndexLayer = []

    CellIndexLayer.append(  FindCellIndex( layer1_x, layer1_y, 'layer1' ) )
    CellIndexLayer.append(  FindCellIndex( layer2_x, layer2_y, 'layer2' ) )
    CellIndexLayer.append(  FindCellIndex( layer3_x, layer3_y, 'layer3' ) )
    CellIndexLayer.append(  FindCellIndex( layer4_x, layer4_y, 'layer4' ) )
    CellIndexLayer.append(  FindCellIndex( layer5_x, layer5_y, 'layer5' ) )
    CellIndexLayer.append(  FindCellIndex( layer6_x, layer6_y, 'layer6' ) )

    return np.array(CellIndexLayer)


def FindCellIndex(tr_x, tr_y, layer_name) : 
        
    x_idx = int( (tr_x + TrDi/2.)/TrDi * layer_struct[layer_name] )
    y_idx = int( (tr_y + TrDi/2.)/TrDi * layer_struct[layer_name] )
    
    return np.array([x_idx, y_idx])




Count number of cells inside the topocluster (Type) in the event (ev) in topocluster number (n)

In [6]:
def Count_Cell(Type,ev, n):
    lay=[64,32,32,16,16,8]
    
    c=0
    for l in range(6):
        if Type=="Total":
            Topo=gT["TopoClusters"+str(l+1)][ev][0]
        
        if Type=="Charged":
            Topo=gCh["TopoClusters"+str(l+1)][ev][0]
            
        for X in range(lay[l]):
            for Y in range(lay[l]):
                
                if Topo[X][Y]==n+1: c+=1
                    
    return c



Compute List of layer's volumes in unit of Radiation length X0

In [7]:
def Vol_Cells():
    
    TrLen=125.
    
    X0_CAL=[3.897,3.897,3.897,2.357,2.357,2.357]
    Lambda_int=[37.578,37.578,37.578,17.438,17.438,17.438]
    layers=[32,64,32,16,16,8]
    Depth=[3*3.897,16*3.897,6*3.897,1.5*17.438,4.1*17.438,1.8*17.438]
    
    X0_tran=[TrLen/X0_CAL[i]/layers[i] for i in range(6)]
    X0_depth=[Depth[i]/X0_CAL[i] for i in range(6)]
        
    V_X0=[]
    for i in range(6):
        V_X0.append(X0_tran[i]*X0_tran[i]*X0_depth[i])
        
        
    return V_X0

In [8]:
print(Vol_Cells())

[3.0142586266295535, 4.019011502172738, 6.028517253259107, 121.92401214091726, 333.2589665185072, 585.2352582764029]


EN_Topo(): Compute array energy with energy (KIND) per topocluster (TOPOCL) in the event

ENN(): Compute Neutral energy predicted by Neural Network within the Topocluster (Total)

In [9]:
#Compute Array with energy (KIND) per topocluster (TOPOCL) in the event     
def En_Topo(KIND, TOPOCL, ev,n):
    layers=[64,32,32,16,16,8]
    En = 0
    EnL = 0
    EnTopo = []
    for i in range(1,7):
        if TOPOCL=="Total":
            Topo = gT['TopoClusters'+str(i)][ev][0]
            Calo = f['RealRes_'+str(KIND)+'Energy_Layer'+str(i)][ev][0]
        if TOPOCL=="Charged":    
            Calo = f['RealRes_'+str(KIND)+'Energy_Layer'+str(i)][ev][0]
            Topo = gCh['TopoClusters'+str(i)][ev][0]
            if n==True:
                Calo+=f['RealRes_Noise_Layer'+str(i)][ev][0]
            
        size=layers[i-1]
            
        TopoA=np.array(Topo[0:size,0:size])
        CaloA=np.array(Calo[0:size,0:size])

        for X in range(len(TopoA)):
            for Y in range(len(TopoA)): 
                TopoI=int(TopoA[X][Y])
                if TopoI!=0:
                    if len(EnTopo)<TopoI: EnTopo.append(Calo[X][Y])
                    else: EnTopo[TopoI-1]=EnTopo[TopoI-1]+CaloA[X][Y]
                        
    return EnTopo



def ENN(n, ev):
    layers=[64,32,32,16,16,8]
    En = 0
    for i in range(1,7):
        Topo = gT['TopoClusters'+str(i)][ev][0]
        Calo = fN['RealRes_NeutralEnergyPred_Layer'+str(i)][ev][0]
        
            
        size=layers[i-1]
            
        TopoA=np.array(Topo[0:size,0:size])
        CaloA=np.array(Calo[0:size,0:size])

        for X in range(len(TopoA)):
            for Y in range(len(TopoA)): 
                TopoI=int(TopoA[X][Y])
                if TopoI==n+1:
                    En+=CaloA[X][Y]
                        
    return En




In [10]:
EV=160
print("Tot_Tot",En_Topo("Total","Total", EV,False))
print("Ch_Tot",En_Topo("Charged","Total", EV,False))
print("Nu_Tot",En_Topo("Neutral","Total", EV,False))
Chaa=En_Topo("Charged","Charged", EV,True)
print("Ch_Ch",Chaa)

#(Chaa[0]-Chaa[1])/(Chaa[0]+Chaa[1])
#ENN(NTot[EV], EV)

Tot_Tot [29480.47879477004, 254.3599054697636, 302.6749994550143]
Ch_Tot [11804.93705577092, 157.74590878521238, 144.46613215843777]
Nu_Tot [15419.1987601044, 0.0, 0.0]
Ch_Ch [13842.455032192034, 254.3599054697636, 302.6749994550143]


# Number of EVENTS

In [11]:
Events=6000

# Layer of High Energy Density (LHED) 

Compute the layer with highest rate of energy density with respecto to depth

## in PI+ sample (Charged)

In [12]:
layers=[64,32,32,16,16,8]

X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

depth=[0, 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]

LHED=[]
for ev in range(Events):
    if ev%100==0: print("ev",ev)
    Vol_Cell=Vol_Cells()
    Dens_En=[]
    Dens_En.append(0)
        
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_indx"]   
    C4=f["Trk_Y_indx"]
    Tr=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev]) 
    
    for lay in range(6):
        Lay_Ev=f["RealRes_ChargedEnergy_Layer"+str(lay+1)][ev][0]+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        
        CCN =gCh["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]

        En_Lay=0
        for X in range(layers[lay]):
            for Y in range(layers[lay]):
                if int(CCN[X][Y])!=0 :
                    ga_we=math.exp(-((Tr[lay][0]-X)**2+(Tr[lay][1]-Y)**2)/(0.00236672*layers[lay]**2))
                    #print(X,Tr[lay][0],Y, Tr[lay][1],layers[lay], ga_we, Lay_Ev[X][Y] )
                    En_Lay+=Lay_Ev[X][Y]/Vol_Cell[lay]*ga_we
        
        Dens_En.append(En_Lay)

    #print("Rho",Dens_En)
    Delta_Dens=[]
    for i in range(1,7):
        Delta_Dens.append((Dens_En[i]-Dens_En[i-1])/(depth[i]-depth[i-1]))
    #print("D",depth)
    #print("DRho",Delta_Dens)

    ind=Delta_Dens.index(max(Delta_Dens))
    #print(ind)
    if Dens_En[ind+1]<Dens_En[ind]: ind-=1
    #print(ind)
    LHED.append(ind)



ev 0
ev 100
ev 200
ev 300
ev 400
ev 500
ev 600
ev 700
ev 800
ev 900
ev 1000
ev 1100
ev 1200
ev 1300
ev 1400
ev 1500
ev 1600
ev 1700
ev 1800
ev 1900
ev 2000
ev 2100
ev 2200
ev 2300
ev 2400
ev 2500
ev 2600
ev 2700
ev 2800
ev 2900
ev 3000
ev 3100
ev 3200
ev 3300
ev 3400
ev 3500
ev 3600
ev 3700
ev 3800
ev 3900
ev 4000
ev 4100
ev 4200
ev 4300
ev 4400
ev 4500
ev 4600
ev 4700
ev 4800
ev 4900
ev 5000
ev 5100
ev 5200
ev 5300
ev 5400
ev 5500
ev 5600
ev 5700
ev 5800
ev 5900


In [13]:
print(LHED[0])

3


## in PI+ + PI0 sample (Total)

In [14]:
layers=[64,32,32,16,16,8]

X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

depth=[0, 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]

LHEDT=[]
for ev in range(Events):
    if ev%100==0: print("ev",ev)
    Vol_Cell=Vol_Cells()
    Dens_En=[]
    Dens_En.append(0)
        
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_indx"]   
    C4=f["Trk_Y_indx"]
    Tr=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev]) 
    
    for lay in range(6):
        Lay_Ev=f["RealRes_TotalEnergy_Layer"+str(lay+1)][ev][0]
        
        CCN =gT["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]

        En_Lay=0
        for X in range(layers[lay]):
            for Y in range(layers[lay]):
                if int(CCN[X][Y])!=0 :
                    ga_we=math.exp(-((Tr[lay][0]-X)**2+(Tr[lay][1]-Y)**2)/(0.00236672*layers[lay]**2))
                    #print(X,Tr[lay][0],Y, Tr[lay][1],layers[lay], ga_we, Lay_Ev[X][Y] )
                    En_Lay+=Lay_Ev[X][Y]/Vol_Cell[lay]*ga_we
        
        Dens_En.append(En_Lay)

    #print("Rho",Dens_En)
    Delta_Dens=[]
    for i in range(1,7):
        Delta_Dens.append((Dens_En[i]-Dens_En[i-1])/(depth[i]-depth[i-1]))
    #print("D",depth)
    #print("DRho",Delta_Dens)

    ind=Delta_Dens.index(max(Delta_Dens))
    #print(ind)
    if Dens_En[ind+1]<Dens_En[ind]: ind-=1
    #print(ind)
    LHEDT.append(ind)




ev 0
ev 100
ev 200
ev 300
ev 400
ev 500
ev 600
ev 700
ev 800
ev 900
ev 1000
ev 1100
ev 1200
ev 1300
ev 1400
ev 1500
ev 1600
ev 1700
ev 1800
ev 1900
ev 2000
ev 2100
ev 2200
ev 2300
ev 2400
ev 2500
ev 2600
ev 2700
ev 2800
ev 2900
ev 3000
ev 3100
ev 3200
ev 3300
ev 3400
ev 3500
ev 3600
ev 3700
ev 3800
ev 3900
ev 4000
ev 4100
ev 4200
ev 4300
ev 4400
ev 4500
ev 4600
ev 4700
ev 4800
ev 4900
ev 5000
ev 5100
ev 5200
ev 5300
ev 5400
ev 5500
ev 5600
ev 5700
ev 5800
ev 5900


In [15]:
print(LHEDT)

[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## Different calculation without track

layers=[64,32,32,16,16,8]

X0_ECAL = 3.897
X0_HCAL = 2.357 

Lambda_int_ECAL = 37.578
Lambda_int_HCAL = 17.438

#Depth in Interaction length
depth=[0, 3*X0_ECAL/Lambda_int_ECAL,16*X0_ECAL/Lambda_int_ECAL,6*X0_ECAL/Lambda_int_ECAL, 1.5, 4.1,1.8 ]


LHED=[]
for ev in range(Events):
    if ev%100==0: print("ev",ev)
    
    Vol_Cell=Vol_Cells()
 
    Dens_En=[]
    
    Dens_En.append(0)
    
    #Approximation topo with most energy in Pi+ sample
    A=En_Topo("Charged","Charged",ev, True)
    if len(A)==0: 
        LHED.append(-1)
        continue
    ind=A.index(max(A))
    A.sort()
    
    #Check of no misunderstandig with the second topocluster E1-E2/E1+E2>0.5
    if len(A)>1 and (A[-1]-A[-2])/(A[-1]+A[-2])<0.5  : 
        LHED.append(-2)
        continue
    
    
    for lay in range(6):
        
        #compute pi+ E/X0^3 in each layer in the most energetic topocluster (leading cluster)
        Lay_Ev=f["RealRes_ChargedEnergy_Layer"+str(lay+1)][ev][0]+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        
        CCN =gCh["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]
        
        En_Lay=0
        for X in range(layers[lay]):
            for Y in range(layers[lay]):
                if int(CCN[X][Y])==ind+1:
                    En_Lay+=Lay_Ev[X][Y]/Vol_Cell[lay]
                    
        Dens_En.append(En_Lay)  
    #print("Rho",Dens_En)
    
    #Compute the greatest variation of rho with respect to depth
    Delta_Dens=[]
    for i in range(1,7):
        Delta_Dens.append((Dens_En[i]-Dens_En[i-1])/(depth[i]-depth[i-1]))
    #print("D",depth)
    #print("DRho",Delta_Dens)
    ind=Delta_Dens.index(max(Delta_Dens))
    #print(ind)
    
    #Choose the layer with greatest E/X0^3
    if Dens_En[ind+1]<Dens_En[ind]: ind-=1
    #print(ind)
    LHED.append(ind)


## Print LHED on png file

In [16]:
hist=ROOT.TH1D("LHED","LHED",6,0,6)
for i in range(Events):
    hist.Fill(LHED[i])
hist.GetXaxis().SetTitle("LHED")
can=ROOT.TCanvas( "LHED","LHED",1600,900);
hist.Draw("")
can.Print("../DATA_PROCESSING/15to20Pred/Pl6000/LHED.png")    

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/15to20Pred/Pl6000/LHED.png has been created


In [17]:
hist1=ROOT.TH1D("LHEDT","LHEDT",6,0,6)
for i in range(Events):
    hist1.Fill(LHEDT[i])
hist1.GetXaxis().SetTitle("LHEDT")
can=ROOT.TCanvas( "LHEDT","LHEDT",1600,900);
hist1.Draw("")
can.Print("../DATA_PROCESSING/15to20Pred/Pl6000/LHEDT.png")    

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/15to20Pred/Pl6000/LHEDT.png has been created


## Compute Ptrack for each event

In [18]:
def P_Tr_Comp(fi):
    Pt=[]
    for i in range(Events):
        En=fi["Smeared_Track_Energy"][i]

        ptr=math.sqrt(En*En-139.57018*139.57018)
        Pt.append(ptr)
    return Pt



P_Track=P_Tr_Comp(f)

## Leading Topocluster in LHED

In [19]:
NCh=[]
layers=[64,32,32,16,16,8]
ODist=[8,4,4,2,2,1]
ODist=[10*x for x in ODist]
l=0
k=0
for ev in range(Events):
    if ev%100==0: print("Ev: "+str(ev))
    
    
    
    #define layer of first interaction
    lay=LHED[ev]
    
    Lay_Ev=f["RealRes_ChargedEnergy_Layer"+str(lay+1)][ev][0]+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        
    Topo_Ev =gCh["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]   
  

    NTopo=len(En_Topo("Charged", "Charged", ev,True))
    
    AverPos=[[0, 0] for i in range(NTopo)]
    E_Topo_Lay=[0 for i in range(NTopo)]
    for X in range(layers[lay]):
        for Y in range(layers[lay]):
            TopoInt=int(Topo_Ev[X][Y])
            if TopoInt!=0:
                #find Center Of Topocluster (AverPos) and
                 
                AverPos[TopoInt-1][0]=AverPos[TopoInt-1][0]+Lay_Ev[X][Y]*X
                AverPos[TopoInt-1][1]=AverPos[TopoInt-1][1]+Lay_Ev[X][Y]*Y
                E_Topo_Lay[TopoInt-1]=E_Topo_Lay[TopoInt-1]+Lay_Ev[X][Y]

    #print(AverPos,E_Topo_Lay, ev, lay)
    
    if E_Topo_Lay==[0]*NTopo:
        l+=1
        NCh.append(-1)
        #print("Skipping ev: ", ev)
        continue
    
    #print(AverPos)
    #print(E_Topo_Lay)
    for j,r in enumerate(AverPos):
        E=E_Topo_Lay[j]
        if E<0.0001: continue
        for i in range(2):
            r[i]=int(r[i]/E_Topo_Lay[j]+0.5)

    #     AverPos Contains mean position of each topocluster
    #print("Center for each clust.: ",AverPos, "", lay+1)
        
        
    #defining track position
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_indx"]   
    C4=f["Trk_Y_indx"]
    Traj_Mark=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev])        
    #print("Real Position of Track: ",Traj_Mark[lay])

        #compute distances Clus-Track
    D=[]
    for r in AverPos:
        D.append(math.sqrt((r[0]-Traj_Mark[lay][0])**2+(r[1]-Traj_Mark[lay][1])**2))
    #print("Distances Clus-Track: ",D)

        #return which clust is nearest
    MinDist=min(D)
    
    #print(MinDist, ODist[lay])
    if MinDist > ODist[lay]:
        k+=1
        NCh.append(-2)
        continue
        
    CN=D.index(MinDist)  
    #print("Cluster nearest to track: ",CN)
        
        #fill a list for each event with number of cluster nearest to track

    NCh.append(CN)

print("too far",k,"no clus", l)

Ev: 0
Ev: 100
Ev: 200
Ev: 300
Ev: 400
Ev: 500
Ev: 600
Ev: 700
Ev: 800
Ev: 900
Ev: 1000
Ev: 1100
Ev: 1200
Ev: 1300
Ev: 1400
Ev: 1500
Ev: 1600
Ev: 1700
Ev: 1800
Ev: 1900
Ev: 2000
Ev: 2100
Ev: 2200
Ev: 2300
Ev: 2400
Ev: 2500
Ev: 2600
Ev: 2700
Ev: 2800
Ev: 2900
Ev: 3000
Ev: 3100
Ev: 3200
Ev: 3300
Ev: 3400
Ev: 3500
Ev: 3600
Ev: 3700
Ev: 3800
Ev: 3900
Ev: 4000
Ev: 4100
Ev: 4200
Ev: 4300
Ev: 4400
Ev: 4500
Ev: 4600
Ev: 4700
Ev: 4800
Ev: 4900
Ev: 5000
Ev: 5100
Ev: 5200
Ev: 5300
Ev: 5400
Ev: 5500
Ev: 5600
Ev: 5700
Ev: 5800
Ev: 5900
too far 0 no clus 0


In [20]:
print(NCh[0])

0


## Num of Cluster without track 

NCh=[]
layers=[64,32,32,16,16,8]

for ev in range(Events):
    if ev%100==0: print("Ev: "+str(ev)) 

    A=En_Topo("Charged","Charged",ev,True)
    if len(A)==0: 
        NCh.append(-1)
        continue
    ind=A.index(max(A))
    A.sort()
    if len(A)>1 and (A[-1]-A[-2])/(A[-1]+A[-2])<0.7  : 
        NCh.append(-2)
        continue
    NCh.append(ind)

## Compute fPred event by event with parametrization in LHED, Pt: fpred(L,Pt)

In [21]:
f_Pred=[]
k=0
for ev in range(Events):
    if ev%100==0: print("Ev: ",ev)
    NTop=NCh[ev]
    if NTop<0: 
        k+=1
        f_Pred.append(-10)
        continue
    ECh=En_Topo("Charged","Charged",ev,True)
    f_Pred.append(ECh[NTop]/P_Track[ev])
    #if ECh[NTop]/P_Track[ev]>1: print(ECh[NTop]/P_Track[ev])
print(k)

Ev:  0
Ev:  100
Ev:  200
Ev:  300
Ev:  400
Ev:  500
Ev:  600
Ev:  700
Ev:  800
Ev:  900
Ev:  1000
Ev:  1100
Ev:  1200
Ev:  1300
Ev:  1400
Ev:  1500
Ev:  1600
Ev:  1700
Ev:  1800
Ev:  1900
Ev:  2000
Ev:  2100
Ev:  2200
Ev:  2300
Ev:  2400
Ev:  2500
Ev:  2600
Ev:  2700
Ev:  2800
Ev:  2900
Ev:  3000
Ev:  3100
Ev:  3200
Ev:  3300
Ev:  3400
Ev:  3500
Ev:  3600
Ev:  3700
Ev:  3800
Ev:  3900
Ev:  4000
Ev:  4100
Ev:  4200
Ev:  4300
Ev:  4400
Ev:  4500
Ev:  4600
Ev:  4700
Ev:  4800
Ev:  4900
Ev:  5000
Ev:  5100
Ev:  5200
Ev:  5300
Ev:  5400
Ev:  5500
Ev:  5600
Ev:  5700
Ev:  5800
Ev:  5900
0


In [22]:
h_Epred= ROOT.TH3D("h_Epred","h_Epred",100,14e3,21e3,6,0,6,50,0.,2.)
h_Epred_prof= ROOT.TProfile2D("h_Epred_prof","h_Epred_prof",100,14e3,21e3,6,0,6,0.,2.)

In [23]:
k=0
for ev in range(Events):
    if NCh[ev]<0 or LHED[ev]<0:
        
        continue
    k+=1
    h_Epred.Fill(P_Track[ev], LHED[ev], f_Pred[ev])
    h_Epred_prof.Fill(P_Track[ev], LHED[ev], f_Pred[ev])
print(k) 

6000


In [24]:
h_Epred_prof.GetXaxis().SetTitle("Param")
can1=ROOT.TCanvas( "Param","Param",1600,900);
#h_Epred_prof.GetXaxis().SetRangeUser(1500,5500);
h_Epred_prof.Draw("colz")
#ROOT.gStyle.SetOptStat(0)
can1.Print("../DATA_PROCESSING/15to20Pred/Pl6000/Param.png")    

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/15to20Pred/Pl6000/Param.png has been created


## Compute fpred as mean of all events 

In [25]:
fp=0
k=0
for ev in range(Events):
    #print(E_Pred[ev])
    if f_Pred[ev]>=0:
        k+=1
        fp+=f_Pred[ev]

fp/=k

print(fp,k)

0.7011757546244788 6000


## Create .root file

f_o=ROOT.TFile("Epred.root","recreate")
h_Epred.Write()
h_Epred_prof.Write()
f_o.Close()

## Compute num topocluster of interaction for total clusters

In [26]:
NTot=[]
layers=[64,32,32,16,16,8]
ODist=[8,4,4,2,2,1]
ODist=[10*x for x in ODist]
l=0
k=0
for ev in range(Events):
    if ev%100==0: print("Ev: "+str(ev))
    
    
    
    #define layer of first interaction
    lay=LHED[ev]
    
    Lay_Ev=f["RealRes_TotalEnergy_Layer"+str(lay+1)][ev][0]#+f["RealRes_Noise_Layer"+str(lay+1)][ev][0]
        
    Topo_Ev =gT["TopoClusters"+str(lay+1)][ev][0][:layers[lay],:layers[lay]]   
  

    NTopo=len(En_Topo("Total", "Total", ev,True))
    
    AverPos=[[0, 0] for i in range(NTopo)]
    E_Topo_Lay=[0 for i in range(NTopo)]
    for X in range(layers[lay]):
        for Y in range(layers[lay]):
            TopoInt=int(Topo_Ev[X][Y])
            if TopoInt!=0:
                #find Center Of Topocluster (AverPos) and
                 
                AverPos[TopoInt-1][0]=AverPos[TopoInt-1][0]+Lay_Ev[X][Y]*X
                AverPos[TopoInt-1][1]=AverPos[TopoInt-1][1]+Lay_Ev[X][Y]*Y
                E_Topo_Lay[TopoInt-1]=E_Topo_Lay[TopoInt-1]+Lay_Ev[X][Y]

    #print(AverPos,E_Topo_Lay, ev, lay)
    
    if E_Topo_Lay==[0]*NTopo:
        l+=1
        NTot.append(-1)
        #print("Skipping ev: ", ev)
        continue
    
    #print(AverPos)
    #print(E_Topo_Lay)
    for j,r in enumerate(AverPos):
        E=E_Topo_Lay[j]
        if E<0.0001: continue
        for i in range(2):
            r[i]=int(r[i]/E_Topo_Lay[j]+0.5)

    #     AverPos Contains mean position of each topocluster
    #print("Center for each clust.: ",AverPos, "", lay+1)
        
        
    #defining track position
    C1=f["Trk_Theta"]    
    C2=f["Trk_Phi"]
    C3=f["Trk_X_indx"]   
    C4=f["Trk_Y_indx"]
    Traj_Mark=MakeTruthTrajectory(C1[ev], C2[ev], C3[ev], C4[ev])        
    #print("Real Position of Track: ",Traj_Mark[lay])

        #compute distances Clus-Track
    D=[]
    for r in AverPos:
        D.append(math.sqrt((r[0]-Traj_Mark[lay][0])**2+(r[1]-Traj_Mark[lay][1])**2))
    #print("Distances Clus-Track: ",D)

        #return which clust is nearest
    MinDist=min(D)
    
    #print(MinDist, ODist[lay])
    if MinDist > ODist[lay]:
        k+=1
        NTot.append(-2)
        continue
        
    CN=D.index(MinDist)  
    #print("Cluster nearest to track: ",CN)
        
        #fill a list for each event with number of cluster nearest to track

    NTot.append(CN)

print("too far",k,"no clus", l)

Ev: 0
Ev: 100
Ev: 200
Ev: 300
Ev: 400
Ev: 500
Ev: 600
Ev: 700
Ev: 800
Ev: 900
Ev: 1000
Ev: 1100
Ev: 1200
Ev: 1300
Ev: 1400
Ev: 1500
Ev: 1600
Ev: 1700
Ev: 1800
Ev: 1900
Ev: 2000
Ev: 2100
Ev: 2200
Ev: 2300
Ev: 2400
Ev: 2500
Ev: 2600
Ev: 2700
Ev: 2800
Ev: 2900
Ev: 3000
Ev: 3100
Ev: 3200
Ev: 3300
Ev: 3400
Ev: 3500
Ev: 3600
Ev: 3700
Ev: 3800
Ev: 3900
Ev: 4000
Ev: 4100
Ev: 4200
Ev: 4300
Ev: 4400
Ev: 4500
Ev: 4600
Ev: 4700
Ev: 4800
Ev: 4900
Ev: 5000
Ev: 5100
Ev: 5200
Ev: 5300
Ev: 5400
Ev: 5500
Ev: 5600
Ev: 5700
Ev: 5800
Ev: 5900
too far 0 no clus 0


## without Use the track information

NTot=[]

for ev in range(Events):    
    if ev%100==0: print("Ev: "+str(ev))

    A=En_Topo("Total","Total",ev,False)
    if len(A)==0: 
        NTot.append(-1)
        continue
    
    ind=A.index(max(A))
    A.sort()
    if len(A)>1 and (A[-1]-A[-2])/(A[-1]+A[-2])<0.7  : 
        NTot.append(-2)
        continue

    NTot.append(ind)

# Results
## Plots of comparison

In [27]:
Distr=[]
String="(E_Pred(L,PT) - E-T):E-T"
#String="(ENeNe - E-T):E-T"
h_EN= ROOT.TH1D(String ,String,100,-1,1)
k=0
p=0
t=0
for ev in range(Events):
    if ev%100==0: print("loop ev: ",ev)
    
    if NTot[ev]<0:
        p+=1
        continue
    if NCh[ev]<0:
        t+=1
        continue


    #Total Topo

    #Neutral
    A=En_Topo("Neutral","Total",ev,False)
    E_N=A[NTot[ev]]

    if E_N==0.0:
        k+=1
        continue

    #Total
    A=En_Topo("Total","Total",ev, False)
    E_Tot=A[NTot[ev]]
    
    #Charged
    #A=En_Topo("Charged","Total",ev, False)
    #E_Ch=A[NTot[ev]]
    
    
    #Neural Network
    ENeNe=ENN(NTot[ev], ev)
    

    
    #Charged Topo
    
    #Charged Noise
    #A=En_Topo("Charged","Charged",ev,True)
    #E_ChN=A[NCh[ev]]
    
    #Charged No Noise
    #A=En_Topo("Charged","Charged",ev,False)
    #E_ChC=A[NCh[ev]]
    
    
    
    
    
    #NumCell
    
    #Total
    #CellT=Count_Cell("Total",ev,NTot[ev])
    
    #Charged
    #CellC=Count_Cell("Charged",ev,NCh[ev])
    
        
    #Parametrization
    EPR=h_Epred_prof.GetBinContent(h_Epred_prof.GetXaxis().FindBin(P_Track[ev]),h_Epred_prof.GetYaxis().FindBin(LHED[ev]))
    

    #h_EN.Fill(fp - E_ChN/P_Track[ev])
    #h_EN.Fill(EPR - E_ChN/P_Track[ev])
    #h_EN.Fill(fp - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill(EPR - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill( ( E_ChN-(E_Tot-E_N) ) / ( E_ChN+(E_Tot-E_N) ) )
    #h_EN.Fill((E_ChC-E_Ch)/(E_ChC+E_Ch))
    #h_EN.Fill(( ((E_ChN-E_ChC) - (E_Tot-E_Ch-E_N)) / ((E_ChN-E_ChC) + (E_Tot-E_Ch-E_N)) ))
    #h_EN.Fill((CellC-CellT)/(CellC+CellT))
    
    k+=1
    h_EN.Fill( ( (E_Tot-EPR*P_Track[ev]) - E_N) /(E_N)    )    
    #h_EN.Fill( (ENeNe- E_N)/E_N )
    
print(fp, t, k, p, Events-p-t-k)

loop ev:  0
loop ev:  100
loop ev:  200
loop ev:  300
loop ev:  400
loop ev:  500
loop ev:  600
loop ev:  700
loop ev:  800
loop ev:  900
loop ev:  1000
loop ev:  1100
loop ev:  1200
loop ev:  1300
loop ev:  1400
loop ev:  1500
loop ev:  1600
loop ev:  1700
loop ev:  1800
loop ev:  1900
loop ev:  2000
loop ev:  2100
loop ev:  2200
loop ev:  2300
loop ev:  2400
loop ev:  2500
loop ev:  2600
loop ev:  2700
loop ev:  2800
loop ev:  2900
loop ev:  3000
loop ev:  3100
loop ev:  3200
loop ev:  3300
loop ev:  3400
loop ev:  3500
loop ev:  3600
loop ev:  3700
loop ev:  3800
loop ev:  3900
loop ev:  4000
loop ev:  4100
loop ev:  4200
loop ev:  4300
loop ev:  4400
loop ev:  4500
loop ev:  4600
loop ev:  4700
loop ev:  4800
loop ev:  4900
loop ev:  5000
loop ev:  5100
loop ev:  5200
loop ev:  5300
loop ev:  5400
loop ev:  5500
loop ev:  5600
loop ev:  5700
loop ev:  5800
loop ev:  5900
0.7011757546244788 0 6000 0 0


In [28]:
h_EN.GetXaxis().SetTitle(String)
can5=ROOT.TCanvas(String,String,1600,900);
h_EN.Draw("")
#ROOT.gStyle.SetOptStat(1)
can5.Print("../DATA_PROCESSING/15to20Pred/Pl6000/"+String+".png")

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/15to20Pred/Pl6000/(E_Pred(L,PT) - E-T):E-T.png has been created


In [29]:
Distr=[]
#String="(E_Pred(L,PT) - E-T):E-T"
String="(ENeNe - E-T):E-T"
h_EN= ROOT.TH1D(String ,String,100,-1,1)
k=0
p=0
t=0
for ev in range(Events):
    if ev%100==0: print("loop ev: ",ev)
    
    if NTot[ev]<0:
        p+=1
        continue
    #if NCh[ev]<0:
    #    t+=1
    #    continue


    #Total Topo

    #Neutral
    A=En_Topo("Neutral","Total",ev,False)
    E_N=A[NTot[ev]]

    if E_N==0.0:
        k+=1
        continue

    #Total
    #A=En_Topo("Total","Total",ev, False)
    #E_Tot=A[NTot[ev]]
    
    #Charged
    #A=En_Topo("Charged","Total",ev, False)
    #E_Ch=A[NTot[ev]]
    
    
    #Neural Network
    ENeNe=ENN(NTot[ev], ev)
    

    
    #Charged Topo
    
    #Charged Noise
    #A=En_Topo("Charged","Charged",ev,True)
    #E_ChN=A[NCh[ev]]
    
    #Charged No Noise
    #A=En_Topo("Charged","Charged",ev,False)
    #E_ChC=A[NCh[ev]]
    
    
    
    
    
    #NumCell
    
    #Total
    #CellT=Count_Cell("Total",ev,NTot[ev])
    
    #Charged
    #CellC=Count_Cell("Charged",ev,NCh[ev])
    
        
    #Parametrization
    EPR=h_Epred_prof.GetBinContent(h_Epred_prof.GetXaxis().FindBin(P_Track[ev]),h_Epred_prof.GetYaxis().FindBin(LHED[ev]))
    

    #h_EN.Fill(fp - E_ChN/P_Track[ev])
    #h_EN.Fill(EPR - E_ChN/P_Track[ev])
    #h_EN.Fill(fp - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill(EPR - (E_Tot-E_N)/P_Track[ev])
    #h_EN.Fill( ( E_ChN-(E_Tot-E_N) ) / ( E_ChN+(E_Tot-E_N) ) )
    #h_EN.Fill((E_ChC-E_Ch)/(E_ChC+E_Ch))
    #h_EN.Fill(( ((E_ChN-E_ChC) - (E_Tot-E_Ch-E_N)) / ((E_ChN-E_ChC) + (E_Tot-E_Ch-E_N)) ))
    #h_EN.Fill((CellC-CellT)/(CellC+CellT))
    
    k+=1
    #h_EN.Fill( ( (E_Tot-EPR*P_Track[ev]) - E_N) /(E_N)    )    
    h_EN.Fill( (ENeNe- E_N)/E_N )
    
print(fp, t, k, p, Events-p-t-k)

loop ev:  0
loop ev:  100
loop ev:  200
loop ev:  300
loop ev:  400
loop ev:  500
loop ev:  600
loop ev:  700
loop ev:  800
loop ev:  900
loop ev:  1000
loop ev:  1100
loop ev:  1200
loop ev:  1300
loop ev:  1400
loop ev:  1500
loop ev:  1600
loop ev:  1700
loop ev:  1800
loop ev:  1900
loop ev:  2000
loop ev:  2100
loop ev:  2200
loop ev:  2300
loop ev:  2400
loop ev:  2500
loop ev:  2600
loop ev:  2700
loop ev:  2800
loop ev:  2900
loop ev:  3000
loop ev:  3100
loop ev:  3200
loop ev:  3300
loop ev:  3400
loop ev:  3500
loop ev:  3600
loop ev:  3700
loop ev:  3800
loop ev:  3900
loop ev:  4000
loop ev:  4100
loop ev:  4200
loop ev:  4300
loop ev:  4400
loop ev:  4500
loop ev:  4600
loop ev:  4700
loop ev:  4800
loop ev:  4900
loop ev:  5000
loop ev:  5100
loop ev:  5200
loop ev:  5300
loop ev:  5400
loop ev:  5500
loop ev:  5600
loop ev:  5700
loop ev:  5800
loop ev:  5900
0.7011757546244788 0 6000 0 0


In [30]:
h_EN.GetXaxis().SetTitle(String)
can5=ROOT.TCanvas(String,String,1600,900);
h_EN.Draw("")
#ROOT.gStyle.SetOptStat(1)
can5.Print("../DATA_PROCESSING/15to20Pred/Pl6000/"+String+".png")

Info in <TCanvas::Print>: png file ../DATA_PROCESSING/15to20Pred/Pl6000/(ENeNe - E-T):E-T.png has been created


In [33]:
Distr=[]
String="(E_Pred(L,PT)-E-T):E-T_vs_LFI "
h= ROOT.TH2D(String ,String,100,-1,1,100,-1,1)
k=0
for ev in range(Events):
    if ev%100==0: print("loop ev: ",ev)
    
    if NTot[ev]<0:
        continue
    if NCh[ev]<0:
        continue
    
    #Total Topo
    
    #Neutral 
    A=En_Topo("Neutral","Total",ev,False)
    E_N=A[NTot[ev]]
    
    #Total
    A=En_Topo("Total","Total",ev, False)
    E_Tot=A[NTot[ev]]
    
    #Charged
    #A=En_Topo("Charged","Total",ev, False)
    #E_Ch=A[NTot[ev]]
    
  
    #Charged Topo
    
    #Charged Noise
    #A=En_Topo("Charged","Charged",ev,True)
    #E_ChN=A[NCh[ev]]
    
    #Charged No Noise
    #A=En_Topo("Charged","Charged",ev,False)
    #E_ChC=A[NCh[ev]]
    
    
    #NumCell
    
    #Total
    CellT=Count_Cell("Total",ev,NTot[ev])
    
    #Charged
    CellC=Count_Cell("Charged",ev,NCh[ev])
    
        
    #Parametrization
    EPR=h_Epred_prof.GetBinContent(h_Epred_prof.GetXaxis().FindBin(P_Track[ev]),h_Epred_prof.GetYaxis().FindBin(LHED[ev]))
    
    if E_N==0: 
        print(ev)
        continue
    k+=1
    h.Fill( ( (E_Tot-EPR*P_Track[ev]) - E_N) /(E_N)  ,(CellC-CellT)/(CellC+CellT))    
    #h_EN.Fill( (E_N -ENNN)/E_N )
    
print(fp, k)

loop ev:  0
33
48
loop ev:  100
101
155
197
loop ev:  200
233
261
loop ev:  300
315
329
384
loop ev:  400
400
488
loop ev:  500
503
523
loop ev:  600
607
646
694
loop ev:  700
700
733
744
772
775
785
788
loop ev:  800
836
loop ev:  900
904
905
914
930
loop ev:  1000
1042
loop ev:  1100
1162
1171
1175
1176
1197
loop ev:  1200
1209
1263
1282
loop ev:  1300
1391
loop ev:  1400
1432
1437
1438
1446
1453
1479
1491
loop ev:  1500
1507
1558
1559
1562
1596
loop ev:  1600
1606
1624
1634
1670
loop ev:  1700
1764
1770
1773
loop ev:  1800
1800
1852
1854
loop ev:  1900
1925
1948
loop ev:  2000
2005
2042
2062
2092
loop ev:  2100
2162
2171
2176
2197
loop ev:  2200
2205
2209
2248
loop ev:  2300
2377
2391
loop ev:  2400
2438
2446
2453
2479
2499
loop ev:  2500
2505
2541
2542
2562
loop ev:  2600
2662
2671
2676
2686
loop ev:  2700
2709
2748
2763
loop ev:  2800
2854
2877
2891
loop ev:  2900
2938
2946
2953
2979
loop ev:  3000
3045
loop ev:  3100
3156
3180
3182
loop ev:  3200
3214
3265
3274
loop ev:  3300
335

Warning in <TROOT::Append>: Replacing existing TH1: (E_Pred(L,PT)-E-T):E-T_vs_LFI  (Potential memory leak).


In [34]:
f_o=ROOT.TFile("Corr.root","recreate")
h.Write()
f_o.Close()